# 如何使用 output-fixing 解析器

这个 [output parser](/docs/concepts/output_parsers/) 包装了另一个 output parser，并且在第一个 parser 失败时，它会调用另一个 LLM 来修复任何错误。

但除了抛出错误，我们还可以做其他事情。具体来说，我们可以将格式错误的输出与格式化指令一起传递给模型，并要求它进行修复。

在此示例中，我们将使用上面的 Pydantic output parser。如果我们传递一个不符合模式的结果给它，会发生以下情况：

In [1]:
from typing import List

from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [2]:
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

In [3]:
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [4]:
try:
    parser.parse(misformatted)
except OutputParserException as e:
    print(e)

Invalid json output: {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE


现在我们可以构建和使用 `OutputFixingParser`。此输Parser接收另一个Parser以及一个 LLM 作为参数，并尝试使用该 LLM 来纠正任何格式错误。

In [5]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [6]:
new_parser.parse(misformatted)

Actor(name='Tom Hanks', film_names=['Forrest Gump'])

查找 [OutputFixingParser](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.fix.OutputFixingParser.html#langchain.output_parsers.fix.OutputFixingParser) 的 API 文档。